In [1]:
import os
import sys

from keras.preprocessing.image import ImageDataGenerator 
# https://keras.io/api/preprocessing/image/#image-data-preprocessing
from keras.applications.imagenet_utils import preprocess_input

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.activations import relu, sigmoid

from keras.optimizers import Adam
from keras.losses import categorical_hinge
from keras.metrics import accuracy

import matplotlib.pyplot as plt

In [2]:
pathTrain = './../dataset/train'
pathTest = './../dataset/test'

if (os.path.exists(pathTrain) and os.path.exists(pathTest)):
    print('OK')
else:
    print('No existen carpetas')

OK


In [3]:
images_increased = 5
width_shape = 224
height_shape = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rotation_range=20, # rota 20 grados
    zoom_range=0.2, # zoom
    width_shift_range=0.1, # desplazar 10%
    height_shift_range=0.1,  # desplazar 10%
    horizontal_flip=True, # invertir horizontalmente
    vertical_flip=False,
    preprocessing_function=preprocess_input
)

valid_datagen = ImageDataGenerator(    
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(  
    pathTrain,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(  
    pathTest,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    class_mode='categorical')

Found 39364 images belonging to 275 classes.
Found 1375 images belonging to 275 classes.


In [4]:
model = Sequential()

inputShape = (height_shape, width_shape, 3)

model.add(Conv2D(filters=33, kernel_size=(3,3), input_shape=inputShape))
model.add(Conv2D(filters=32, kernel_size=(3,3)))
model.add(MaxPool2D())

model.add(Conv2D(filters=64, kernel_size=(3,3)))
model.add(Conv2D(filters=64, kernel_size=(3,3)))
model.add(MaxPool2D())

model.add(Flatten())

model.add(Dense(64, activation=relu))
model.add(Dense(23, activation=relu))
model.add(Dense(1, activation=sigmoid))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 33)      924       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 32)      9536      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 179776)            0

In [5]:
adam = Adam(learning_rate=0.001)

model.compile(
    optimizer=adam,
    loss=categorical_hinge,
    metrics=[accuracy]
)

In [6]:
epoch = 5

history = model.fit(
    train_generator,
    steps_per_epoch=5,
    validation_data=validation_generator,
    epochs=epoch
)

Epoch 1/5
 16/100 [===>..........................] - ETA: 3:44 - loss: 1.0000 - accuracy: 0.0127

KeyboardInterrupt: 

In [ ]:
history.history
plt.plot(range(len(history.history['loss'])), history.history['loss'], color='red')
plt.plot(range(len(history.history['val_loss'])), history.history['val_loss'], color='blue')

plt.show()